## This notebook is part of self learning Quantum Computing Algorithms

## Description: This program optimizes the stock portfolio
### With real time data + increase in assests + increase in budget + increase in num of days

In [71]:
import numpy as np
from qiskit_finance.data_providers import RandomDataProvider
from pandas_datareader import data
import datetime
import pandas as pd

In [74]:
num_assets = 6
seed = 123
start=datetime.datetime(2019,1,1)
end=datetime.datetime(2021,6,30)
# Generate expected return and covariance matrix from (random) time-series
stocks = ['AAPL', 'NKE', 'GOOGL', 'AMZN', 'FB', 'MSFT']

In [75]:
# Import data
df = data.DataReader(stocks, 'yahoo', start=start, end=end)

In [76]:
df.head()

Attributes  Adj Close                                                   \
Symbols          AAPL        NKE        GOOGL         AMZN          FB   
Date                                                                     
2019-01-02  38.382229  72.298897  1054.680054  1539.130005  135.679993   
2019-01-03  34.559078  71.020050  1025.469971  1500.280029  131.740005   
2019-01-04  36.034370  72.874878  1078.069946  1575.390015  137.949997   
2019-01-07  35.954170  73.919434  1075.920044  1629.510010  138.050003   
2019-01-08  36.639565  74.905426  1085.369995  1656.579956  142.529999   

Attributes                 Close                                       ...  \
Symbols          MSFT       AAPL        NKE        GOOGL         AMZN  ...   
Date                                                                   ...   
2019-01-02  97.961319  39.480000  74.059998  1054.680054  1539.130005  ...   
2019-01-03  94.357529  35.547501  72.750000  1025.469971  1500.280029  ...   
2019-01-04  98.746010  37.064999  74.650002  1078.069946  1575.390015  ...   
2019-01-07  98.871956  36.982498  75.720001  1075.920044  1629.510010  ...   
2019-01-08  99.588829  37.687500  76.730003  1085.369995  1656.579956  ...   

Attributes         Open                                            Volume  \
Symbols           GOOGL         AMZN          FB        MSFT         AAPL   
Date                                                                        
2019-01-02  1027.199951  1465.199951  128.990005   99.550003  148158800.0   
2019-01-03  1050.670044  1520.010010  134.690002  100.099998  365248800.0   
2019-01-04  1042.560059  1530.000000  134.009995   99.720001  234428400.0   
2019-01-07  1080.969971  1602.310059  137.559998  101.639999  219111200.0   
2019-01-08  1086.000000  1664.689941  139.889999  103.040001  164101200.0   

Attributes                                                           
Symbols           NKE      GOOGL       AMZN          FB        MSFT  
Date                                                                 
2019-01-02  6762700.0  1593400.0  7983100.0  28146200.0  35329300.0  
2019-01-03  8007400.0  2098000.0  6975600.0  22717900.0  42579100.0  
2019-01-04  7844200.0  2301100.0  9182600.0  29002100.0  44060600.0  
2019-01-07  8184800.0  2372300.0  7993200.0  20089300.0  35656100.0  
2019-01-08  8809000.0  1770700.0  8881400.0  26263800.0  31514400.0  

[5 rows x 36 columns]

In [77]:
df = df['Adj Close']

In [78]:
df.head()

Symbols,AAPL,NKE,GOOGL,AMZN,FB,MSFT
Date,,,,,,
2019-01-02,38.382229,72.298897,1054.680054,1539.130005,135.679993,97.961319
2019-01-03,34.559078,71.020050,1025.469971,1500.280029,131.740005,94.357529
2019-01-04,36.034370,72.874878,1078.069946,1575.390015,137.949997,98.746010
2019-01-07,35.954170,73.919434,1075.920044,1629.510010,138.050003,98.871956
2019-01-08,36.639565,74.905426,1085.369995,1656.579956,142.529999,99.588829


In [79]:
df_data = []
for stock in stocks:
    df_data.append(df[stock].values.tolist())
print(df_data)

[[38.38222885131836, 34.559078216552734, 36.03437042236328, 35.95417022705078, 36.639564514160156, 37.26177215576172, 37.38086700439453, 37.01386260986328, 36.4572868347168, 37.20344161987305, 37.65793991088867, 37.88154220581055, 38.114871978759766, 37.25934600830078, 37.410030364990234, 37.11351776123047, 38.34333038330078, 37.98849105834961, 37.59475326538086, 40.16376876831055, 40.453006744384766, 40.472450256347656, 41.6220588684082, 42.334197998046875, 42.348777770996094, 41.54671859741211, 41.59553527832031, 41.356327056884766, 41.71270751953125, 41.53939437866211, 41.690738677978516, 41.59797286987305, 41.72246551513672, 41.990966796875, 41.75419616699219, 42.22041702270508, 42.5279655456543, 42.55237579345703, 42.68418502807617, 42.26435089111328, 42.70859909057617, 42.92338943481445, 42.84527587890625, 42.5987548828125, 42.105690002441406, 42.20576477050781, 43.66787338256836, 44.15849685668945, 44.353763580322266, 44.84682083129883, 45.43020248413086, 45.89398193359375, 45.5

## $\min\limits_{x \in \{0, 1\}^{n}} q \ x^{T}\Sigma x − \mu^{T}x \\ \text{or} \\ \max\limits_{x \in \{0, 1\}^{n}} \mu^{T}x - q \ x^{T}\Sigma x \\ \text{subject to}: 1^{T} x = B$

##### $\text{ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html} \\  x \in \{0, 1\}^{n} \ \text{denotes the vector of binary decision variables, which indicate which assets to pick} \left(x[i]=1\right) \text{and which not to pick} \left(x[i]=0\right), \\ \mu \in R^{n} \ \text{defines the expected returns for the assets}, \\ \Sigma \in R^{n \times n} \text{specifies the covariances between the assets}, \\ q \gt 0 \ \text{controls the risk appetite of the decision maker}, \\ \text{and B denotes the budget, i.e. the number of assets to be selected out of n.}$

## $\text{The equality constraint} \ 1^{T}x = B \ \text{is mapped to a penalty term} \ \left(1^{T} x − B\right)^{2}$

## $\text{Assumptions}:\\ \ \cdot \text{all assets have the same price} \left(\text{normalized to 1}\right) \\ \ \cdot \text{the full budget B has to be spent, i.e., one has to select exactly B assets.}$

In [22]:
def divide_2(val_1, val_2):
    if val_2 == 0:
        if val_1 == 0:
            return 1
        return np.nan
    return val_1 / val_2

## Calculate mean returns 
## $\mu \in R^{n} \ \text{defines the expected returns for the assets}$

In [5]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(data)[:, 1:], np.array(data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## Calculate mean covariance
##  $\Sigma \in R^{n \times n} \text{specifies the covariances between the assets}$

In [49]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
sigma = np.cov(period_returns)

## plot sigma

In [27]:
import matplotlib.pyplot as plt

In [48]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## $q \gt 0 \ \text{controls the risk appetite of the decision maker}$

In [24]:
q = 0.5 # set risk factor

## $\text{B denotes the budget, i.e. the number of assets to be selected out of n.}$

In [60]:
budget = 3 # set budget (B)

### ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html

In [61]:
penalty = num_assets # set parameter to scale the budget penalty term

## Model

In [29]:
from docplex.mp.model import Model

In [62]:
mdl = Model('portfolio model')
#x = mdl.binary_var_list(num_assets)

In [63]:
x = list()
for i in range(num_assets):
    x.append(mdl.binary_var(name="x_{0}".format(i)))
print(x)

[docplex.mp.Var(type=B,name='x_0'), docplex.mp.Var(type=B,name='x_1'), docplex.mp.Var(type=B,name='x_2'), docplex.mp.Var(type=B,name='x_3'), docplex.mp.Var(type=B,name='x_4'), docplex.mp.Var(type=B,name='x_5')]


## $\max\limits_{x \in \{0, 1\}}  \mu^{T} * x - q * x^{T} * \Sigma * x \\ \text{linear} = c^{T}x, \text{qudratic} = x^{T}Qx = \sum\limits_{i, j = 1}^{n} x_{i}^{T} \Sigma_{ij} x_{j} \\ \text{objective} = \text{linear - }\left(\text{risk_factor * qudratic}\right)$

In [64]:
linear = np.dot(mu, x) # mu^T * x
qudratic = mdl.sum([x[i]*sigma[i][j]*x[j] for i in range(num_assets) for j in range(num_assets)])
objective = linear - q * qudratic
mdl.maximize(objective)

## budget constraint:  $\sum\limits_{i =1}^{n} 1^{T}x_{i} == \text{budget}$

In [65]:
cost = mdl.sum([x[i] for i in range(num_assets)])
mdl.add_constraint(cost == budget, ctname='budget')

docplex.mp.LinearConstraint[budget](x_0+x_1+x_2+x_3+x_4+x_5,EQ,3)

## converting to Quadratic Program

## removing the constraint to create the QUBO

In [35]:
from qiskit_optimization.translators import from_docplex_mp

In [66]:
mod = from_docplex_mp(mdl)

In [67]:
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio model

Maximize
 obj: 0.004682261762 x_0 + 0.007159640330 x_1 + 0.001227757962 x_2
      + 0.003234622966 x_3 + 0.002725277591 x_4 + 0.004364434718 x_5 + [
      - 0.000095526386 x_0^2 + 0.000075990183 x_0*x_1 - 0.000091374658 x_0*x_2
      - 0.000085351262 x_0*x_3 - 0.000138108962 x_0*x_4 - 0.000097158652 x_0*x_5
      - 0.001244762113 x_1^2 + 0.000012725679 x_1*x_2 + 0.000326101317 x_1*x_3
      + 0.000149750656 x_1*x_4 + 0.000144573046 x_1*x_5 - 0.000063470332 x_2^2
      - 0.000047494852 x_2*x_3 - 0.000129234354 x_2*x_4 - 0.000111986840 x_2*x_5
      - 0.000126959488 x_3^2 - 0.000100278531 x_3*x_4 - 0.000069931804 x_3*x_5
      - 0.000226327311 x_4^2 - 0.000182207269 x_4*x_5 - 0.000071725030 x_5^2 ]/2
Subject To
 budget: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 = 3

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
End



## QAOA

In [38]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit import Aer

In [68]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

In [69]:
def print_result(result):
    selection = result.x
    res = list()
    for i in range(len(selection)):
        if selection[i]:
            res.append(stocks[i])
    value = result.fval
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))
    print('Optimal: selected stocks {}'.format(res))

    eigenstate = result.min_eigen_solver_result.eigenstate
    eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = QuadraticProgramToQubo().convert(mod).objective.evaluate(x)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [70]:
algorithm_globals.random_seed = 1234
backend = Aer.get_backend('statevector_simulator')
cobyla = COBYLA()
cobyla.set_options(maxiter=250)
quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
qaoa_mes = QAOA(optimizer=cobyla, reps=3, quantum_instance=quantum_instance)
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa_mes)
result = eigen_optimizer.solve(mod)
print_result(result)

Optimal: selection [1. 1. 0. 0. 0. 1.], value 0.0156
Optimal: selected stocks ['AAPL', 'NKE', 'MSFT']

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[0 0 1 1 1 0]	-0.0068		0.0506
[0 0 1 0 1 1]	-0.0079		0.0504
[1 0 1 0 1 0]	-0.0083		0.0504
[0 0 1 1 0 1]	-0.0086		0.0503
[1 0 1 1 0 0]	-0.0089		0.0503
[0 0 0 1 1 1]	-0.0099		0.0501
[1 0 1 0 0 1]	-0.0100		0.0501
[1 0 0 1 1 0]	-0.0103		0.0501
[0 1 1 0 1 0]	-0.0104		0.0500
[0 1 1 1 0 0]	-0.0111		0.0499
[1 0 0 0 1 1]	-0.0114		0.0499
[1 0 0 1 0 1]	-0.0120		0.0498
[0 1 1 0 0 1]	-0.0121		0.0498
[1 1 1 0 0 0]	-0.0124		0.0497
[0 1 0 1 1 0]	-0.0125		0.0497
[0 1 0 0 1 1]	-0.0135		0.0495
[1 1 0 0 1 0]	-0.0138		0.0495
[0 1 0 1 0 1]	-0.0142		0.0494
[1 1 0 1 0 0]	-0.0145		0.0494
[1 1 0 0 0 1]	-0.0156		0.0492
[1 1 1 1 1 1]	9.2044		0.0001
[0 0 1 0 0 0]	4.0996		0.0001
[0 0 0 0 1 0]	4.0981		0.0001
[0 0 0 1 0 0]	4.0976		0.0001
[0 0 0 0 0 1]	4.0964		0.0001
[1 0 0 0 0 0]	4.096

## If you have any queries on this notebook please reach to me bala.na@hcl.com